# **Exploratory Data Analysis**


## Importing libraries

In [1]:
import pandas as pd
pd.options.display.max_columns=20
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px


In [2]:
from google.colab import drive
drive.mount('/content/drive')
sales = pd.read_csv('/content/drive/Shareddrives/Capstone/CAPSTONE MODEL/datosArticuloLocalizacionS.csv', sep = ';')
items = pd.read_csv('/content/drive/Shareddrives/Capstone/CAPSTONE MODEL/datosArticuloS.csv', sep = ',')
place = pd.read_csv('/content/drive/Shareddrives/Capstone/CAPSTONE MODEL/datosLocalizacionS.csv', sep = ',')

MessageError: ignored

## Data exploration

In [ ]:
sales.info()

In [ ]:
place.info()

In [ ]:
items.info()

In [ ]:
# merging the 3 datasets to obtain a single dataframe
df = sales.merge(items,how = 'left' , on='COD_ARTICULO').merge(place, how = 'left', on = 'COD_LOCALIZACION')

In [ ]:
df = df.sort_values(by ='FECHA_PRIMERA_VENTA').reset_index().drop('index', axis=1)
df['FECHA']= df['FECHA_PRIMERA_VENTA']

df['FECHA'] = pd.to_datetime(df['FECHA'], infer_datetime_format=True)  

df['YEAR'] = pd.DatetimeIndex(df['FECHA']).year
df['MONTH'] = pd.DatetimeIndex(df['FECHA']).month
df['DAY'] = pd.DatetimeIndex(df['FECHA']).day

In [ ]:
df.info()

## Explore missing values


In [ ]:
import missingno as msno

In [ ]:
# Variable of number of missing values per column
num_missing_values = df.isna().sum()

# See where there actually are missing values
print(num_missing_values[num_missing_values != 0])

# See what type of data the columns with missing values have
print("\n")
print(df[pd.DataFrame(num_missing_values[num_missing_values != 0]).index.tolist()].dtypes)

In [ ]:
df.isnull().sum()

In [ ]:
sns.heatmap(data = df.isnull(), annot = False, yticklabels = False, cbar = False, cmap = 'viridis')

In [ ]:
msno.bar(df, color='purple')

In [ ]:
pd.DataFrame(df.fillna('miss').where(df[['MERCADO', 'LATITUD', 'LONGITUD']]=='miss')).value_counts

## Distribution of variables

In [ ]:
# TARGET VARIABLE: UDS_VENDIDAS_TOTAL
# set the plot size
plt.figure(figsize=(10,6))


# set the histogram, mean and median
sns.distplot(df["UDS_VENDIDAS_TOTAL"], kde=False)
plt.axvline(df["UDS_VENDIDAS_TOTAL"].mean(), linewidth=3, color='g', label="mean", alpha=0.5)
plt.axvline(df["UDS_VENDIDAS_TOTAL"].median(), linewidth=3, color='y', label="median", alpha=0.5)

# set title, legends and labels
plt.xlabel("Total sales")
plt.ylabel("Count")
plt.title("Distribution of UDS_VENDIDAS_TOTAL", size=14)
plt.legend(["mean", "median"])

In [ ]:
# VARIABLE: FACTURADO, Numerical
# set the plot size
plt.figure(figsize=(10,6))


# set the histogram, mean and median
sns.distplot(df["FACTURADO"], kde=False)
plt.axvline(df["FACTURADO"].mean(), linewidth=3, color='g', label="mean", alpha=0.5)
plt.axvline(df["FACTURADO"].median(), linewidth=3, color='y', label="median", alpha=0.5)

# set title, legends and labels
plt.xlabel("Total Inventory")
plt.ylabel("Count")
plt.title("Distribution of FACTURADO", size=14)
plt.legend(["mean", "median"])

In [ ]:
# VARIABLE: Campaign Duration, Numeric

0# set the plot size
plt.figure(figsize=(10,6))


# set the histogram, mean and median
sns.distplot(df["DIAS_HASTA_SALDO"], kde=False)
plt.axvline(df["DIAS_HASTA_SALDO"].mean(), linewidth=3, color='g', label="mean", alpha=0.5)
plt.axvline(df["DIAS_HASTA_SALDO"].median(), linewidth=3, color='y', label="median", alpha=0.5)

# set title, legends and labels
plt.xlabel("Campaign duration")
plt.ylabel("Count")
plt.title("Distribution of campaign duration", size=14)
plt.legend(["mean", "median"])

## Groupings

In [ ]:
color_counts = df['COD_COLOR'].value_counts().head(15)

fig, ax = plt.subplots(figsize=(10, 6))
plt.rcParams.update({'font.size': 12})
# Plot a bar chart of the color counts
color_counts.plot(kind='bar')

# Add a title and axis labels
plt.title('Color Counts')
plt.xlabel('Color')
plt.ylabel('Count')

# Show the plot
plt.show()

In [ ]:

# Group the data by country and color and count the number of occurrences
color_counts = df.groupby(['MERCADO', 'COD_COLOR']).size().reset_index(name='count')

# For each country, find the color with the highest count
most_popular_colors = color_counts.loc[color_counts.groupby('MERCADO')['count'].idxmax()]

# Print the most popular color for each country
print(most_popular_colors)

In [ ]:
# Group the data by country and color and count the number of occurrences
color_counts = df.groupby(['MERCADO', 'COD_COLOR']).size().reset_index(name='count')

# Sort the data by country and count
sorted_color_counts = color_counts.sort_values(['MERCADO', 'count'], ascending=[True, False])

# Get the top 3 colors for each country
top_5_colors = sorted_color_counts.groupby('MERCADO').head(5)

# Print the top 3 colors for each country
print(top_5_colors)

In [ ]:
# Group the data by country and color and count the number of occurrences
color_counts = df.groupby(['MERCADO', 'COD_COLOR']).size().reset_index(name='count')

# Sort the data by country and count in ascending order
sorted_color_counts = color_counts.sort_values(['MERCADO', 'count'], ascending=[True, True])

# Get the worst 5 colors for each country
worst_5_colors = sorted_color_counts.groupby('MERCADO').head(5)

# Print the worst 5 colors for each country
print(worst_5_colors)

## Questions

### Does price affect sales?

In [ ]:
plt.scatter(df['PRECIO'], df['UDS_VENDIDAS_TOTAL'])

# Add a title and axis labels
plt.title('Price vs Sales')
plt.xlabel('Price')
plt.ylabel('Total Units Sold')

# Show the plot
plt.show()

### Does campaign duration affect sales?

In [ ]:
plt.scatter(df['DIAS_HASTA_SALDO'], df['UDS_VENDIDAS_TOTAL'])

# Add a title and axis labels
plt.title('Duration vs Sales')
plt.xlabel('Duration')
plt.ylabel('Units Sold')

# Show the plot
plt.show()

In [ ]:
# Count the number of occurrences of each value in the MERCADO column
mercado_counts = df['MERCADO'].value_counts()

# Plot a bar chart of the mercado counts
mercado_counts.plot(kind='bar')

# Add a title and axis labels
plt.title('Mercado Counts')
plt.xlabel('Mercado')
plt.ylabel('Count')

# Show the plot
plt.show()

In [ ]:
# VARIABLE: COD_TIPO_LOCALIZACION, CATEGORICAL
# set the plot size
plt.figure(figsize=(10,6))


# set the histogram, mean and median
#sns.hist(df["COD_TIPO_LOCALIZACION"])
#sns.distplot(df["COD_TIPO_LOCALIZACION"], kde=False)
#plt.axvline(df["COD_TIPO_LOCALIZACION"].mean(), linewidth=3, color='g', label="mean", alpha=0.5)
#plt.axvline(df["COD_TIPO_LOCALIZACION"].median(), linewidth=3, color='y', label="median", alpha=0.5)
fig = px.histogram(df, x="COD_TIPO_LOCALIZACION", title='Campaigns per COD_TIPO_LOCALIZACION')
fig.update_layout(title_text='Campaigns per type of location', title_x=0.5)


# set title, legends and labels
#3plt.xlabel("COD_TIPO_LOCALIZACION")
#plt.ylabel("# campaigns per type of location")
#plt.title("Distribution of UDS_VENDIDAS_TOTAL", size=14)
#plt.legend(["mean", "median"])
fig.show()


In [ ]:
print(df['COD_TIPO_LOCALIZACION'].value_counts()/df['COD_TIPO_LOCALIZACION'].value_counts().sum())
print('')
print('T4 and T7 are minority classes, with each <=5% of all observations')
print('T4 and T7 will be put in a single class called minority')

In [ ]:
# VARIABLE: COD_COMPRADOR, CATEGORICAL
# set the plot size
plt.figure(figsize=(10,6))


# set the histogram, mean and median
#sns.hist(df["COD_TIPO_LOCALIZACION"])
#sns.distplot(df["COD_TIPO_LOCALIZACION"], kde=False)
#plt.axvline(df["COD_TIPO_LOCALIZACION"].mean(), linewidth=3, color='g', label="mean", alpha=0.5)
#plt.axvline(df["COD_TIPO_LOCALIZACION"].median(), linewidth=3, color='y', label="median", alpha=0.5)
fig = px.histogram(df, x="COD_COMPRADOR", title='Campaigns per COD_COMPRADOR')
fig.update_layout(title_text='Campaigns per type of buyer', title_x=0.5)


# set title, legends and labels
#3plt.xlabel("COD_TIPO_LOCALIZACION")
#plt.ylabel("# campaigns per type of location")
#plt.title("Distribution of UDS_VENDIDAS_TOTAL", size=14)
#plt.legend(["mean", "median"])
fig.show()


In [ ]:
print(df['COD_COMPRADOR'].value_counts()/df['COD_COMPRADOR'].value_counts().sum())
print('')
print('No minority classes, we keep it as it is')


In [ ]:
# VARIABLE: COD_TIPO, CATEGORICAL
# set the plot size
plt.figure(figsize=(10,6))


# set the histogram, mean and median
#sns.hist(df["COD_TIPO_LOCALIZACION"])
#sns.distplot(df["COD_TIPO_LOCALIZACION"], kde=False)
#plt.axvline(df["COD_TIPO_LOCALIZACION"].mean(), linewidth=3, color='g', label="mean", alpha=0.5)
#plt.axvline(df["COD_TIPO_LOCALIZACION"].median(), linewidth=3, color='y', label="median", alpha=0.5)
fig = px.histogram(df, x="COD_TIPO", title='Campaigns per COD_TIPO')
fig.update_layout(title_text='Campaigns per type of article', title_x=0.5)


# set title, legends and labels
#3plt.xlabel("COD_TIPO_LOCALIZACION")
#plt.ylabel("# campaigns per type of location")
#plt.title("Distribution of UDS_VENDIDAS_TOTAL", size=14)
#plt.legend(["mean", "median"])
fig.show()


In [ ]:
print(df['COD_TIPO'].value_counts()/df['COD_TIPO'].value_counts().sum())
print('')
print('The classes with less than 5% observations, will be put together into a minority class')


In [ ]:
# VARIABLE: COD_SUBTIPO, CATEGORICAL
# set the plot size
plt.figure(figsize=(10,6))


# set the histogram, mean and median
#sns.hist(df["COD_TIPO_LOCALIZACION"])
#sns.distplot(df["COD_TIPO_LOCALIZACION"], kde=False)
#plt.axvline(df["COD_TIPO_LOCALIZACION"].mean(), linewidth=3, color='g', label="mean", alpha=0.5)
#plt.axvline(df["COD_TIPO_LOCALIZACION"].median(), linewidth=3, color='y', label="median", alpha=0.5)
fig = px.histogram(df, x="COD_SUBTIPO", title='Campaigns per COD_SUBTIPO')
fig.update_layout(title_text='Campaigns per sub type of article', title_x=0.5)


# set title, legends and labels
#3plt.xlabel("COD_TIPO_LOCALIZACION")
#plt.ylabel("# campaigns per type of location")
#plt.title("Distribution of UDS_VENDIDAS_TOTAL", size=14)
#plt.legend(["mean", "median"])
fig.show()


In [ ]:
print(df['COD_SUBTIPO'].value_counts()/df['COD_SUBTIPO'].value_counts().sum())
print('')
print('The classes with less than 1% observations, will be put together into a minority class')


In [ ]:
# VARIABLE: MERCADO, CATEGORICAL
# set the plot size
plt.figure(figsize=(10,6))


# set the histogram, mean and median
#sns.hist(df["COD_TIPO_LOCALIZACION"])
#sns.distplot(df["COD_TIPO_LOCALIZACION"], kde=False)
#plt.axvline(df["COD_TIPO_LOCALIZACION"].mean(), linewidth=3, color='g', label="mean", alpha=0.5)
#plt.axvline(df["COD_TIPO_LOCALIZACION"].median(), linewidth=3, color='y', label="median", alpha=0.5)
fig = px.histogram(df, x="MERCADO", title='Campaigns per MERCADO')
fig.update_layout(title_text='Campaigns per MERCADO', title_x=0.5)


# set title, legends and labels
#3plt.xlabel("COD_TIPO_LOCALIZACION")
#plt.ylabel("# campaigns per type of location")
#plt.title("Distribution of UDS_VENDIDAS_TOTAL", size=14)
#plt.legend(["mean", "median"])
fig.show()
print('Mostly Spanish Campaigns.We will not use binning as it is important for the purpose of this project to keep countries as they are')


## Correlation between variables

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
sns.heatmap(df.corr())

In [ ]:
## how does facturado affect uds vendidas?
plt.figure(figsize=(6,6))
plt.scatter(x='FACTURADO',y='UDS_VENDIDAS_TOTAL',data=df)
plt.xlabel('FACTURADO')
plt.ylabel('UDS VENDIDAS')

In [ ]:
df1 = pd.read_csv('preprocessed_data.csv', sep = ',').drop('Unnamed: 0', axis=1)

In [ ]:
df1.info()

## Sales by year



In [ ]:
## Sales by year
a = df1.groupby('year')[['UDS_VENDIDAS_TOTAL']].sum().reset_index()
# create a bar chart using plotly express
fig = px.bar(a, x='year', y='UDS_VENDIDAS_TOTAL')

# update the chart layout
fig.update_layout(
    title='Units Sold Per Year',
    xaxis_title='Year',
    yaxis_title='Units Sold'
)

# show the chart
fig.show()

## Sales by country

In [ ]:
a = df1.groupby('MERCADO')[['UDS_VENDIDAS_TOTAL']].sum().reset_index()

In [ ]:
# create a bar chart using plotly express
fig = px.bar(a, x='MERCADO', y='UDS_VENDIDAS_TOTAL')

# update the chart layout
fig.update_layout(
    title='Units Sold Per Country',
    xaxis_title='Country',
    yaxis_title='Units Sold'
)
fig.update_layout(title_text='Units Sold Per Country', title_x=0.5)

# show the chart
fig.show()

## DIAS_HASTA_SALDO vs Total Units Sold 

In [ ]:
a = df1.groupby(['DIAS_HASTA_SALDO'])[['UDS_VENDIDAS_TOTAL']].sum().reset_index()

In [ ]:
# create a bar chart using plotly express
fig = px.bar(a, x='DIAS_HASTA_SALDO', y='UDS_VENDIDAS_TOTAL')

# update the chart layout
fig.update_layout(
    title='Units Sold',
    xaxis_title='DIAS_HASTA_SALDO in campaign',
    yaxis_title='Units Sold'
)
fig.update_layout(title_text='Units Sold vs Length of Campaign', title_x=0.5)


# show the chart
fig.show()

In [ ]:
df1.columns

In [ ]:
# create a bar chart using plotly express
fig = px.bar(df1, x='PRECIO', y='UDS_VENDIDAS_TOTAL')

# update the chart layout
fig.update_layout(
    title='Units Sold',
    xaxis_title='autumn days in campaign',
    yaxis_title='Units Sold'
)

# show the chart
fig.show()